In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pandas as pd
with open('/Users/xiangyikong/Desktop/week73/quantile_model_for_flu/data/sensor_graph/adj_mx.pkl', 'rb') as f:
    states_data = pickle.load(f)

In [2]:
states_in_the_prediction = ['01',
 '02',
 '04',
 '05',
 '06',
 '08',
 '09',
 '10',
 '12',
 '13',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '53',
 '54',
 '55',
 '56']

### Week 15
#### Forecast weeks: week16,week17,week18,week19
#### Forecast date: 2022-06-11, 2022-06-18, 2022-06-25, 2022-07-02

In [ ]:
npz = np.load("/Users/xiangyikong/Desktop/week73/quantile_model_for_flu/plotweek15result/seed8epo277.npz")
pred = npz['prediction']
truth = npz['truth']
pred_reshaped = pred.reshape(50, 4 ,3)

In [ ]:
median_prediction = []
for i in range(50):
    median_prediction.append(pred_reshaped[i,:,1])

In [ ]:
dfg = pd.read_csv("/Users/xiangyikong/Desktop/week73/truth-Incident Hospitalizations.csv")
dfg['date'] = pd.to_datetime(dfg['date'])
dfg_date_filtered = dfg[(dfg['date'] >= '2022-06-11') & (dfg['date'] <= '2022-07-02')]

In [ ]:
dfg_date_filtered['date'].unique()

In [ ]:
dfg_date_filtered_truth_filtered = dfg_date_filtered[dfg_date_filtered['location'].isin(states_in_the_prediction)]
dfg_date_filtered_truth_filtered = dfg_date_filtered_truth_filtered.sort_values(by=['date', 'location'])
dfg_date_filtered_truth_filtered

In [ ]:
states = dfg_date_filtered_truth_filtered['location_name'].unique()

In [ ]:
median_prediction = np.array(median_prediction)
actual_number_perdiction = median_prediction + np.resize(dfg_date_filtered_truth_filtered['value'].to_numpy(), (4,50)).T

In [ ]:
actual_number_perdiction

In [ ]:
np.resize(dfg_date_filtered_truth_filtered['value'].to_numpy(), (4,50)).T

#### Quantile Model

In [ ]:
def get_plots_quantile_prediction(npz_file_location):
    npz = np.load(npz_file_location)
    pred = npz['prediction']
    # truth = npz['truth']
    ground_truth = np.resize(dfg_date_filtered_truth_filtered['value'].to_numpy(), (4,50)).T
    # print(pred.shape,truth.shape)
    # if "quantile_model" in npz_file_location:
    #     pred_reshaped = pred.reshape(50, 4 ,3)
    #     truth_reshaped = truth.reshape(50, 4)
    # elif "dropout_model" in npz_file_location:
    #     pred_reshaped = pred.reshape(50, 4)
    #     truth_reshaped = truth.reshape(50, 4)
    pred_reshaped = pred.reshape(50, 4 ,3)

    # truth_reshaped = truth.reshape(50, 4)

    x = np.arange(1,5)
    # states = list(states_data[1])
    states = dfg_date_filtered_truth_filtered['location_name'].unique()
    for i in range(50):
        plt.plot(x, pred_reshaped[i,:,1] + ground_truth[i,:],label="Prediction")
        plt.plot(x, pred_reshaped[i,:,0] + ground_truth[i,:],label="Lower Bound")
        plt.plot(x, pred_reshaped[i,:,2] + ground_truth[i,:],label="Upper Bound")
        plt.plot(x, ground_truth[i,:],label="Truth")
        plt.title(states[i])
        plt.xlabel("Num of Weeks Ahead")
        plt.ylabel("Number of Flu Cases")
        plt.legend(loc='best')
        plt.fill_between(x, pred_reshaped[i,:,0] + ground_truth[i,:],pred_reshaped[i,:,2] + ground_truth[i,:],color = 'green',alpha = 0.2)
        plt.show()

In [ ]:
get_plots_quantile_prediction("/Users/xiangyikong/Desktop/week73/quantile_model_for_flu/plotweek15result/seed8epo277.npz")

In [ ]:
def get_plots_quantile(npz_file_location,test_sets):
    npz = np.load(npz_file_location)
    pred = npz['prediction'].transpose((1,0,2,3))[test_sets]
    # pred = npz['prediction']

    truth = npz['truth'].transpose((1,0,2))[test_sets]
    # ground_truth = np.resize(dfg_date_filtered_truth_filtered['value'].to_numpy(), (4,50)).T
    # print(pred.shape,truth.shape)
    # if "quantile_model" in npz_file_location:
    #     pred_reshaped = pred.reshape(50, 4 ,3)
    #     truth_reshaped = truth.reshape(50, 4)
    # elif "dropout_model" in npz_file_location:
    #     pred_reshaped = pred.reshape(50, 4)
    #     truth_reshaped = truth.reshape(50, 4)
    pred_reshaped = pred.reshape(50, 4 ,3)
    truth_reshaped = truth.reshape(50, 4)

    x = np.arange(1,5)
    states = list(states_data[1])
    states = dfg_date_filtered_truth_filtered['location_name'].unique()
    for i in range(50):
        plt.plot(x, pred_reshaped[i,:,1],label="Prediction")
        plt.plot(x, pred_reshaped[i,:,0],label="Lower Bound")
        plt.plot(x, pred_reshaped[i,:,2],label="Upper Bound")
        plt.plot(x, truth_reshaped[i,:],label="Truth")
        plt.title(states[i])
        plt.xlabel("Num of Weeks Ahead")
        plt.ylabel("Residual between Predicted and True")
        plt.legend(loc='best')
        plt.fill_between(x, pred_reshaped[i,:,0],pred_reshaped[i,:,2] ,color = 'green',alpha = 0.2)
        plt.show()

In [ ]:
get_plots_quantile("/Users/xiangyikong/Desktop/week73/quantile_model_for_flu/plotweek15_unshuffledresult/seed8epo247.npz",0)

In [ ]:
get_plots_quantile("/Users/xiangyikong/Desktop/week73/quantile_model_for_flu/plotweek15_unshuffledresult/seed8epo247.npz",1)

In [ ]:
get_plots_quantile("/Users/xiangyikong/Desktop/week73/quantile_model_for_flu/plotweek15_unshuffledresult/seed8epo247.npz",2)

#### Spline Quantile Model

In [ ]:
low_prediction = np.load("/Users/xiangyikong/Desktop/DeepGLEAM/sq_model/model/pytorch/low_prediction.npy")
median_prediction = np.load("/Users/xiangyikong/Desktop/DeepGLEAM/sq_model/model/pytorch/median_prediction.npy")
high_prediction = np.load("/Users/xiangyikong/Desktop/DeepGLEAM/sq_model/model/pytorch/high_prediction.npy")
truth = np.load("/Users/xiangyikong/Desktop/DeepGLEAM/sq_model/model/pytorch/groundtruth.npy")

In [ ]:
sq_preds = np.stack([low_prediction,median_prediction,high_prediction]).transpose((3,2,1,0)).reshape(50,4,3)
# np.stack([low_prediction,median_prediction,high_prediction]).transpose((3,2,1,0)).reshape(50,4,3)
sq_truth = truth.reshape(50,4)

In [ ]:
print(sq_truth.shape,
sq_preds.shape)

In [ ]:
x = np.arange(1,5)
states = list(states_data[1])
for i in range(50):
    plt.plot(x, sq_preds[i,:,1],label = "Prediction")
    plt.plot(x, sq_preds[i,:,0],label = "Lower Bound")
    plt.plot(x, sq_preds[i,:,2],label = "Upper Bound")
    plt.plot(x, sq_truth[i,:],label = "Truth")
    plt.title(states[i])
    plt.xlabel("Num of Weeks Ahead")
    plt.ylabel("Residual between Predicted and True")
    plt.legend(loc='best')
    plt.show()

#### Dropout Model

In [ ]:
loaded = []
for i in range(0,20):
    temp = np.load("/Users/xiangyikong/Desktop/DeepGLEAM/dropout_model/plotweek31result/seed9/" + str(i) + ".npz")
    loaded.append(temp)

In [ ]:
loaded[0]['truth']

In [ ]:
prediction_lst = []
for i in range(0,20):
    prediction_lst.append(loaded[i]['prediction'])
truth_lst = []
for i in range(0,20):
    truth_lst.append(loaded[i]['truth'])

In [ ]:
average_of_20_truth = np.mean(np.stack(truth_lst,axis = 0),axis=0).transpose((1,2,0)).reshape(50,4)

In [ ]:
average_of_20_truth

In [ ]:
np.stack(prediction_lst).shape

In [ ]:
average_of_20 = np.mean(np.stack(prediction_lst),axis=0)

In [ ]:
std_pred = np.std(np.stack(prediction_lst),axis=0)

In [ ]:
lower_bound_pred = average_of_20 - 1.96*(std_pred)
upper_bound_pred = average_of_20 + 1.96*(std_pred)
# Replaced with mean -1.96std and mean + 1.96std
# quantile_0_975 = np.quantile(np.stack(prediction_lst),0.975,axis=0)

In [ ]:
x = np.arange(1,5)
preds = np.stack([lower_bound_pred,average_of_20,upper_bound_pred]).transpose((3,1,2,0)).reshape(50,4,3)
truth = np.mean(np.stack(truth_lst,axis = 0),axis=0).transpose((1,2,0)).reshape(50,4)
states = list(states_data[1])
for i in range(50):
    plt.plot(x, preds[i,:,1],label = "Prediction")
    plt.plot(x, preds[i,:,0],label = "Lower Bound")
    plt.plot(x, preds[i,:,2],label = "Upper Bound")
    plt.plot(x, truth[i,:],label = "Truth")
    plt.title(states[i])
    plt.xlabel("Num of Weeks Ahead")
    plt.ylabel("Residual between Predicted and True")
    plt.legend(loc='best')
    plt.show()

#### Week 34 Quantile Model

In [ ]:
import pickle


with open('/Users/xiangyikong/Desktop/DeepGLEAM/quantile_model/data/sensor_graph/adj_mx.pkl', 'rb') as f:
    states_data = pickle.load(f)

In [ ]:
x = np.arange(1,5)
states = list(states_data[1])
for i in range(50):
    plt.plot(x, seed9epo311_prediction_reshaped[i,:,1])
    plt.plot(x, seed9epo311_prediction_reshaped[i,:,0])
    plt.plot(x, seed9epo311_prediction_reshaped[i,:,2])
    plt.plot(x, seed9epo311_truth_reshaped[i,:])
    plt.title(states[i])
    plt.xlabel("Num of Weeks Ahead")
    plt.ylabel("Residual between Predicted and True")
    plt.show()